# Data Load

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

train_bookmark = pd.read_csv('Train/train_bookmark.csv', parse_dates=['dates'], infer_datetime_format=True)
train_service = pd.read_csv('Train/train_service.csv', parse_dates=['registerdate','enddate'], infer_datetime_format=True)
coin = pd.read_csv('Code/coin.csv')
content_info = pd.read_csv('Code/content_info.csv')
movie_info = pd.read_csv('Code/movie_info.csv')
predict_bookmark = pd.read_csv('Predict/predict_bookmark.csv', parse_dates=['dates'], infer_datetime_format=True)
predict_service = pd.read_csv('Predict/predict_service.csv', parse_dates=['registerdate','enddate'], infer_datetime_format=True)

In [3]:
train_bookmark.tail()

,uno,dates,hour,channeltype,programid,title,contentid,contentnumber,section,viewtime,devicetype
7824107,998384efe9466eb3fed96243f6a6831de4deda30bb8dee...,2021-03-21,18,L,S01,NaN,NaN,NaN,NaN,1900,5
7824108,998384efe9466eb3fed96243f6a6831de4deda30bb8dee...,2021-03-20,18,L,M01,NaN,NaN,NaN,NaN,1170,5
7824109,998384efe9466eb3fed96243f6a6831de4deda30bb8dee...,2021-03-30,23,L,K04,NaN,NaN,NaN,NaN,3590,5
7824110,998384efe9466eb3fed96243f6a6831de4deda30bb8dee...,2021-03-31,22,L,C2501,NaN,NaN,NaN,NaN,1820,5
7824111,998384efe9466eb3fed96243f6a6831de4deda30bb8dee...,2021-03-31,23,L,C2501,NaN,NaN,NaN,NaN,3130,5


# 전처리

## 결측치 처리

In [2]:
exchange_rate = 1138.5

train_service['promo_100'] = train_service['promo_100'].apply(lambda x: 0 if pd.isna(x) else 1 )
train_service['coinReceived'] = train_service['coinReceived'].apply(lambda x: 0 if pd.isna(x) else 1 )
train_service['isauth'] = train_service['isauth'].apply(lambda x: 0 if pd.isna(x) else 1 )
train_service['Repurchase'] = train_service['Repurchase'].apply(lambda x: 1 if x=='X' else 0)
train_service['gender'] = train_service['gender'].fillna('N')
train_service['pgamount'] = train_service['pgamount'].apply(lambda x: x * exchange_rate if x<100 else x)

predict_service['promo_100'] = predict_service['promo_100'].apply(lambda x: 0 if pd.isna(x) else 1 )
predict_service['coinReceived'] = predict_service['coinReceived'].apply(lambda x: 0 if pd.isna(x) else 1 )
predict_service['isauth'] = predict_service['isauth'].apply(lambda x: 0 if pd.isna(x) else 1 )
# predict_service['Repurchase'] = predict_service['Repurchase'].apply(lambda x: int(1) if x=='X' else int(np.nan))
predict_service['gender'] = predict_service['gender'].fillna('N')
predict_service['pgamount'] = predict_service['pgamount'].apply(lambda x: x * exchange_rate if x<100 else x)

In [ ]:
train_service[train_service.enddate >= train_service.registerdate + pd.DateOffset(weeks=3)]

In [ ]:
predict_service[predict_service.enddate >= predict_service.registerdate + pd.DateOffset(weeks=3)]

# EDA

In [12]:
# train_service.groupby('uno')['registerdate'].count().sort_values(ascending=False)
train_service[train_service['uno'] == 'a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c362622171c550dcfe7099733a95d986e379eedfcb8edb5adc79c3f2439a56104f63410b83a10131c8ea5'].sort_values(by='registerdate')

,uno,registerdate,enddate,productcode,pgamount,chargetypeid,concurrentwatchcount,promo_100,coinReceived,Repurchase,devicetypeid,isauth,gender,agegroup
41285,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-02 20:26:09,2021-03-04 14:35:46,pk_1488,10900.0,180,2,0,0,1,pc,0,N,40
61977,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-04 15:35:46,2021-03-05 08:03:44,pk_1487,7900.0,180,1,0,0,1,pc,0,N,40
32509,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-05 09:03:44,2021-03-07 14:17:04,pk_1488,10900.0,180,2,0,0,1,pc,0,N,40
55219,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-07 15:17:04,2021-03-08 14:43:20,pk_1487,7900.0,180,1,0,0,1,pc,0,N,40
8517,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-08 14:43:20,2021-03-09 22:29:36,pk_1488,10900.0,180,2,0,0,1,pc,0,N,40
3391,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-09 22:29:36,2021-03-12 23:06:30,pk_1487,7900.0,180,1,0,0,1,pc,0,N,40
237,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-12 23:06:30,2021-04-12 23:06:30,pk_1488,10900.0,180,2,0,0,1,pc,0,N,40


In [15]:
train_bookmark[train_bookmark['uno'] == 'a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c362622171c550dcfe7099733a95d986e379eedfcb8edb5adc79c3f2439a56104f63410b83a10131c8ea5'].sort_values(by=['dates', 'hour'])

,uno,dates,hour,channeltype,programid,title,contentid,contentnumber,section,viewtime,devicetype
27044,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-02,23,V,M_T66101G,제2공화국,M_EP201901307390.1,40,1,70,1
27045,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-02,23,V,M_T66101G,제2공화국,M_EP201901307391.1,41,1,70,1
27046,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-04,16,V,K01_T1999-0038,왕과 비,K01_PS-T1999-0038-1-000.1,1,1,1330,1
27043,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-06,22,V,M_1000018100000100000,왕초,M_1000018100028100000.1,28,1,140,1
27036,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-13,0,V,S01_V0000341172,제중원,S01_10000142104.1,23,1,70,1
27026,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-13,4,V,M_1000018100000100000,왕초,M_1000018100002100000.1,2,1,7200,1
27027,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-13,5,V,M_1000018100000100000,왕초,M_1000018100003100000.1,3,1,7200,1
27031,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-13,5,V,M_1000018100000100000,왕초,M_1000018100002100000.1,2,1,7200,1
27033,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-13,6,V,M_1000018100000100000,왕초,M_1000018100003100000.1,3,1,7200,1
27042,a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c36...,2021-03-13,6,V,M_1000018100000100000,왕초,M_1000018100004100000.1,4,1,7200,1


In [ ]:
pd.DataFrame(train_service.groupby(['chargetypeid', 'Repurchase'])['chargetypeid'].count())

In [45]:
train_service[train_service['uno'] == '734a2bd8e4a4d878fd9663aeff2f83317de5fc668079bf7909f56663edc08550da8380e30b96b53617f36c622407380e0373f6017fded']

,uno,registerdate,enddate,productcode,pgamount,chargetypeid,concurrentwatchcount,promo_100,coinReceived,Repurchase,devicetypeid,isauth,gender,agegroup,view_count,viewtime_sum,viewtime_mean,viewtime_std


In [ ]:
train_service.groupby('Repurchase')['pgamount'].mean()

In [ ]:
train_service['promo_100'].unique()

In [ ]:
predict_service.head()

In [ ]:
coin.head()

In [ ]:
content_info.head()

In [ ]:
movie_info.head()

# Feature 추가

## 가입기간별 View Info

In [2]:
for i in train_service.index:
    df = train_bookmark[(train_bookmark['uno'] == train_service.loc[i, 'uno']) & (train_bookmark['dates'] >= train_service.loc[i, 'registerdate']) & (train_bookmark['dates'] < train_service.loc[i, 'enddate'])]
#     print(df.groupby('uno')['dates'].count()[0], df.groupby('uno')['viewtime'].sum()[0], df.groupby('uno')['viewtime'].mean()[0], df.groupby('uno')['viewtime'].std()[0])
    if len(df) > 0:
        train_service.loc[i, 'view_count'] = df.groupby('uno')['dates'].count()[0]
        train_service.loc[i, 'viewtime_sum'] = df.groupby('uno')['viewtime'].sum()[0]
        train_service.loc[i, 'viewtime_mean'] = df.groupby('uno')['viewtime'].mean()[0]
        train_service.loc[i, 'viewtime_std'] = df.groupby('uno')['viewtime'].std()[0]
    else:
        pass
    
train_service.head()

KeyboardInterrupt: 

In [ ]:
for i in predict_service.index:
    df = predict_bookmark[(train_bookmark['uno'] == predict_service.loc[i, 'uno']) & (predict_bookmark['dates'] >= predict_service.loc[i, 'registerdate']) & (predict_bookmark['dates'] < predict_service.loc[i, 'enddate'])]
    if len(df) > 0:
        predict_service.loc[i, 'view_count'] = df.groupby('uno')['dates'].count()[0]
        predict_service.loc[i, 'viewtime_sum'] = df.groupby('uno')['viewtime'].sum()[0]
        predict_service.loc[i, 'viewtime_mean'] = df.groupby('uno')['viewtime'].mean()[0]
        predict_service.loc[i, 'viewtime_std'] = df.groupby('uno')['viewtime'].std()[0]
    else:
        pass
    
predict_service.head()

In [ ]:
# train_service = pd.merge(train_service, 
#          pd.DataFrame(train_bookmark.groupby('uno')['contentid'].count()).rename(columns={'contentid':'view_count'}).reset_index(),
#          on='uno', how='left')
# train_service = pd.merge(train_service, 
#          pd.DataFrame(train_bookmark.groupby('uno')['viewtime'].sum()).rename(columns={'viewtime':'viewtime_sum'}).reset_index(),
#          on='uno', how='left')
# train_service = pd.merge(train_service, 
#          pd.DataFrame(train_bookmark.groupby('uno')['viewtime'].mean()).rename(columns={'viewtime':'viewtime_mean'}).reset_index(),
#          on='uno', how='left')
# train_service = pd.merge(train_service, 
#          pd.DataFrame(train_bookmark.groupby('uno')['viewtime'].std()).rename(columns={'viewtime':'viewtime_std'}).reset_index(),
#          on='uno', how='left')
train_service['view_count'].fillna(0, inplace=True)
train_service['viewtime_sum'].fillna(0, inplace=True)
train_service['viewtime_mean'].fillna(0, inplace=True)
train_service['viewtime_std'].fillna(0, inplace=True)

# predict_service = pd.merge(predict_service, 
#          pd.DataFrame(predict_bookmark.groupby('uno')['contentid'].count()).rename(columns={'contentid':'view_count'}).reset_index(),
#          on='uno', how='left')
# predict_service = pd.merge(predict_service, 
#          pd.DataFrame(predict_bookmark.groupby('uno')['viewtime'].sum()).rename(columns={'viewtime':'viewtime_sum'}).reset_index(),
#          on='uno', how='left')
# predict_service = pd.merge(predict_service, 
#          pd.DataFrame(predict_bookmark.groupby('uno')['viewtime'].mean()).rename(columns={'viewtime':'viewtime_mean'}).reset_index(),
#          on='uno', how='left')
# predict_service = pd.merge(predict_service, 
#          pd.DataFrame(predict_bookmark.groupby('uno')['viewtime'].std()).rename(columns={'viewtime':'viewtime_std'}).reset_index(),
#          on='uno', how='left')
predict_service['view_count'].fillna(0, inplace=True)
predict_service['viewtime_sum'].fillna(0, inplace=True)
predict_service['viewtime_mean'].fillna(0, inplace=True)
predict_service['viewtime_std'].fillna(0, inplace=True)

## Productcode별 해지율

In [ ]:
rateByproductcode = pd.DataFrame(train_service.groupby(['productcode', 'Repurchase'])['Repurchase'].count()) / pd.DataFrame(train_service.groupby(['productcode'])['Repurchase'].count())
rateByproductcode.columns = ['RepurchaseRateByProductcode']
rateByproductcode.reset_index(inplace=True)

In [ ]:
train_service = pd.merge(train_service, 
                 rateByproductcode[rateByproductcode['Repurchase'] == 1][['productcode', 'RepurchaseRateByProductcode']], 
                 on='productcode')


predict_service = pd.merge(predict_service, 
                 rateByproductcode[rateByproductcode['Repurchase'] == 1][['productcode', 'RepurchaseRateByProductcode']], 
                 on='productcode')
predict_service.head()

## chargetypeid별 해지율

In [ ]:
rateByChargetypeid = pd.DataFrame(train_service.groupby(['chargetypeid', 'Repurchase'])['Repurchase'].count()) / pd.DataFrame(train_service.groupby(['chargetypeid'])['Repurchase'].count())
rateByChargetypeid.columns = ['rateByChargetypeid']
rateByChargetypeid.reset_index(inplace=True)

In [ ]:
rateByChargetypeid.head()

In [ ]:
train_service = pd.merge(train_service, 
                 rateByChargetypeid[rateByChargetypeid['Repurchase'] == 1][['chargetypeid', 'rateByChargetypeid']], 
                 on='chargetypeid')

predict_service = pd.merge(predict_service, 
                 rateByChargetypeid[rateByChargetypeid['Repurchase'] == 1][['chargetypeid', 'rateByChargetypeid']], 
                 on='chargetypeid')
predict_service.head()

In [ ]:
train_service.corr()

In [ ]:
predict_service.drop('rateByChargetypeid_x', axis=1, inplace=True)
predict_service.drop('rateByChargetypeid_y', axis=1, inplace=True)

## Device Count

In [ ]:
train_service = pd.merge(train_service, 
                 train_bookmark.groupby('uno')['devicetype'].nunique(), 
                 on='uno')

train_service.rename(columns={'devicetype':'devicetype_cnt'}, inplace=True)

predict_service = pd.merge(predict_service, 
                 predict_bookmark.groupby('uno')['devicetype'].nunique(), 
                 on='uno')
predict_service.rename(columns={'devicetype':'devicetype_cnt'}, inplace=True)

## 고객별 가입이력

In [ ]:
pd.DataFrame(train_service.groupby('uno')['registerdate'].count()).sort_values(by='registerdate', ascending=False)

In [ ]:
train_service[train_service['uno']=='a02a14ff1ab86edbfe46ad5a6a7fce054dc83c39aa7c362622171c550dcfe7099733a95d986e379eedfcb8edb5adc79c3f2439a56104f63410b83a10131c8ea5']

## 시리즈물 시청 정보

In [ ]:
train_bookmark.head()

In [ ]:
content_info[content_info['contentid'].str.contains('S01_E454434902')]

In [ ]:
movie_info.head()

# Feature Select

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.figure(figsize=(18,10))
sns.heatmap(train_service.corr(), annot=True)   

In [ ]:
train_columns = list(train_service.columns)
train_columns.remove('Repurchase')
train_columns

# Model

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(train_service[train_columns].select_dtypes(['float64', 'int64']), 
                                                    train_service['Repurchase'], 
                                                    test_size=0.2, 
                                                    stratify=train_service['Repurchase'])

scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

params = {
    'n_neighbors' : range(1, 21, 1)   
}
clf = KNeighborsClassifier()
grid_cv = GridSearchCV(clf, param_grid=params, scoring='f1', cv=5, verbose=1)
grid_cv.fit(X_train_norm, y_train)

# clf.fit(X_train, y_train)
pred = grid_cv.best_estimator_.predict(X_test_norm)
print(grid_cv.best_score_)
print(grid_cv.best_params_)
print(classification_report(y_test, pred))


In [ ]:
y_train.shape

## Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(train_service[train_columns].select_dtypes(['float64', 'int64']), 
                                                    train_service['Repurchase'], 
                                                    test_size=0.2, 
                                                    stratify=train_service['Repurchase'])

params = {
    'max_depth' : range(1, 21, 2),
    'min_samples_split' : range(1, 11, 1)    
}
clf = DecisionTreeClassifier()
grid_cv = GridSearchCV(clf, param_grid=params, scoring='f1', cv=5, verbose=1)
grid_cv.fit(X_train, y_train)

# clf.fit(X_train, y_train)
pred = grid_cv.best_estimator_.predict(X_test)
print(grid_cv.best_score_)
print(grid_cv.best_params_)
print(classification_report(y_test, pred))

In [ ]:
pred_submission = clf.predict(predict_service[train_columns].select_dtypes(['float64', 'int64']))
predict_service['pred'] = pred_submission
predict_service.head()

## RandomForestClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(train_service[train_columns].select_dtypes(['float64', 'int64']), 
                                                    train_service['Repurchase'], 
                                                    test_size=0.2, 
                                                    stratify=train_service['Repurchase'])

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

rcf = RandomForestClassifier()
rcf.fit(X_train, y_train)
pred = rcf.predict(X_test)
print(classification_report(y_test, pred))

In [ ]:
pred_submission = rcf.predict(predict_service[train_columns].select_dtypes(['float64', 'int64']))
predict_service['pred'] = pred_submission
predict_service.head()

## Randomforest GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(train_service[train_columns].select_dtypes(['float64', 'int64']), 
                                                    train_service['Repurchase'], 
                                                    test_size=0.2, 
                                                    stratify=train_service['Repurchase'])

params = {
    'n_estimators': [300], #list(range(100, 501, 100)),
#     'max_features' : [int(np.sqrt(len(train_columns)))],
    'max_depth' : [6], #range(1, 10, 1),
    'min_samples_leaf': [5], #range(1, 11, 1),
    'min_samples_split': [2] #list(range(1, 11, 1))
}

rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=5, n_jobs=-1, scoring='f1')
grid_cv.fit(X_train, y_train)
pred = grid_cv.best_estimator_.predict(X_test)
print(grid_cv.best_score_)
print(grid_cv.best_params_)
print(classification_report(y_test, pred))

In [ ]:
pred_submission = grid_cv.predict(predict_service[train_columns].select_dtypes(['float64', 'int64']))
predict_service['pred'] = pred_submission
predict_service.head()

## SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = train_test_split(train_service[train_columns].select_dtypes(['float64', 'int64']), 
                                                    train_service['Repurchase'], 
                                                    test_size=0.2, 
                                                    stratify=train_service['Repurchase'])

svm = SVC(kernel='rbf', gamma=0.10, C=10.0)
svm.fit(X_train, y_train)
pred = svm.predict(X_test)
print(classification_report(y_test, pred))

## XGBClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = train_test_split(train_service[train_columns].select_dtypes(['float64', 'int64']), 
                                                    train_service['Repurchase'], 
                                                    test_size=0.2, 
                                                    stratify=train_service['Repurchase'])

# xgb_wrapper = XGBRFClassifier(n_estimators=400, learning_rate=0.1, max_depth=15)
# xgb_wrapper.fit(X_train, y_train)
# pred = xgb_wrapper.predict(X_test)

xgb_wrapper = XGBClassifier(objective = 'binary:logistic')
params = {
     'n_estimators': [300],
    'learning_rate' : [0.1],
    'max_depth': [20],
    'min_samples_split': [1, 2, 3, 4, 5]
}

grid_cv = GridSearchCV(xgb_wrapper, param_grid=params, cv=5, n_jobs=-1, scoring='f1')
grid_cv.fit(X_train, y_train)
pred = grid_cv.best_estimator_.predict(X_test)
print(grid_cv.best_score_)
print(grid_cv.best_params_)
print(classification_report(y_test, pred))

## XGBRFClassifier

### 1

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRFClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = train_test_split(train_service[train_columns].select_dtypes(['float64', 'int64']), 
                                                    train_service['Repurchase'], 
                                                    test_size=0.2, 
                                                    stratify=train_service['Repurchase'])

# xgb_wrapper = XGBRFClassifier(n_estimators=400, learning_rate=0.1, max_depth=15)
# xgb_wrapper.fit(X_train, y_train)
# pred = xgb_wrapper.predict(X_test)

xgb_wrapper = XGBRFClassifier(objective = 'binary:logistic')
params = {
     'n_estimators': [300],
    'learning_rate' : [0.1],
    'max_depth': [20],
    'min_samples_split': [1, 2, 3, 4, 5]
}

grid_cv = GridSearchCV(xgb_wrapper, param_grid=params, cv=5, n_jobs=-1, scoring='f1')
grid_cv.fit(X_train, y_train)
pred = grid_cv.best_estimator_.predict(X_test)
print(grid_cv.best_score_)
print(grid_cv.best_params_)
print(classification_report(y_test, pred))

In [ ]:
pred_submission = xgb_wrapper.predict(predict_service[train_columns].select_dtypes(['float64', 'int64']))
predict_service['pred'] = pred_submission
predict_service.head()

# Submission

In [ ]:
# 결과 제출 답안지 불러오기
# predict_service['Churn'] = predict_service['pred'].apply(lambda x: 0 if x==1 else 1)
predict_service['Churn'] = predict_service['pred']

ds_sheet = "CDS_submission.csv"
df_sheet = pd.read_csv(ds_sheet)
df_sheet.drop('CHURN', axis=1, inplace=True)
df_sheet.info()

# 답안지에 답안 표기
df_result = predict_service.loc[:,('uno','registerdate','productcode')]
df_result['KEY']   = df_result['uno'] + '|' + df_result['registerdate'].dt.strftime('%y-%m-%d %I:%M:%S') + '|' + df_result['productcode']   # 판다스 strftime()
df_result['CHURN'] = predict_service['Churn']
df_result = df_result.loc[:,('KEY','CHURN')]
df_answer_sheet = pd.merge(df_sheet, df_result, on='KEY', how='left')


# 답안지 제출 파일 생성하기
ds_answer_sheet = "CDS_submission_RF_RepurchaseRateByProductcode.csv"
df_answer_sheet.to_csv(ds_answer_sheet, index=False, encoding='utf8')

In [ ]:
predict_service.head()

In [ ]:
df_answer_sheet

In [ ]:
predict_service[predict_service['uno'] == '7de6c80c6cb5c5098bbfef8d9da75dfdd338b681d96691b420cbddbf35b2cb878a07272fd1a3d9eb2506a19f27738f05dfc8627da765152ccc1a44e565b4a86e|21-03-14 10:55:38|pk_2025']

In [ ]:
predict_service[predict_service['uno'] == '7de6c80c6cb5c5098bbfef8d9da75dfdd338b681d96691b420cbddbf35b2cb878a07272fd1a3d9eb2506a19f27738f05dfc8627da765152ccc1a44e565b4a86e']

In [ ]:
predict_service.head(10)

In [ ]:
predict_service.shape